# Collaboration

## Configure environments

In [1]:
# Run this to change the working directory.
# TODO: pip install will resolve this issue...
import os
os.chdir("..")

In [2]:
import numpy as np
import torch
from imagiq.federated.nodes import Node
from imagiq.models import Model
from imagiq.datasets import CBISDDSMDataset, LoadBreastDensity, LoadBreastDensityd
from imagiq.utils.file_systems import remove, mkdir
from monai.transforms import (
    Compose,
    LoadImaged,
    ScaleIntensityd,
    SqueezeDimd,
    AddChanneld,
    AsChannelFirstd,
    Lambdad,
    ToTensord,
    Resized,
    RandRotated,
    RandFlipd,
    RandHistogramShiftd,
    RandGaussianNoised,
    RandZoomd,
    RepeatChanneld, 
    NormalizeIntensityd
)
from monai.networks.nets import densenet121, densenet169, se_resnet50, se_resnet101, se_resnet152
from monai.data import CacheDataset
import sys
import pandas as pd
import gc

## Load Dataset

In [3]:
# Transformation for breast density dataset
train_transform = Compose( [
    LoadImaged( keys='image'),
    Lambdad(keys='image', func=lambda x: x.T),
    AsChannelFirstd('image'),
    RepeatChanneld('image', repeats=3),
    Resized('image', spatial_size=(225,225), mode='nearest'),
    ScaleIntensityd('image'), # scale data to 0~1
    NormalizeIntensityd( 'image', 
                        subtrahend=[0.485, 0.456, 0.406],
                        divisor=[0.229, 0.224, 0.225], 
                        channel_wise=True), # standardize with ImageNet weights
    RandFlipd('image', spatial_axis=0, prob=0.5), 
    RandZoomd( 'image', min_zoom=0.9, max_zoom=1.5, prob=0.5, keep_size=True),
    ToTensord( ('image', 'label') ),
])

val_transform = Compose( [
    LoadImaged( keys='image'),
    Lambdad(keys='image', func=lambda x: x.T),
    AsChannelFirstd('image'),
    RepeatChanneld('image', repeats=3),
    Resized('image', spatial_size=(225,225), mode='nearest'),
    ScaleIntensityd('image'),
    NormalizeIntensityd( 'image', 
                        subtrahend=[0.485, 0.456, 0.406],
                        divisor=[0.229, 0.224, 0.225], 
                        channel_wise=True), # standardize with ImageNet weights
    ToTensord( ('image', 'label') ),
])


train_ds = CBISDDSMDataset( section='training', transforms=train_transform, download=[0])
val_ds = CBISDDSMDataset( section='validation', transforms=val_transform, download=[0])
test_ds = CBISDDSMDataset( section='test', transforms=val_transform, download=[0])

cbisddsm.csv: 16.0kB [00:01, 8.41kB/s]                            
images_001.zip: 0.00B [00:00, ?B/s]


downloaded file: /Users/navya/.imagiq/datasets/CBISDDSM/cbisddsm.csv.
Expected md5 is None, skip md5 check for file /Users/navya/.imagiq/datasets/CBISDDSM/cbisddsm.csv.


images_001.zip: 3.81GB [01:55, 35.5MB/s]                                



downloaded file: /Users/navya/.imagiq/datasets/CBISDDSM/images_001.zip.
Expected md5 is None, skip md5 check for file /Users/navya/.imagiq/datasets/CBISDDSM/images_001.zip.
Expected md5 is None, skip md5 check for file /Users/navya/.imagiq/datasets/CBISDDSM/images_001.zip.
Update [0008|103e]: Calc-Test_P_00127_RIGHT_MLO.dcm
Update [0020|0060]: Calc-Test_P_00127_RIGHT_MLO.dcm
Update [0008|103e]: Calc-Test_P_00127_RIGHT_CC.dcm
Update [0020|0060]: Calc-Test_P_00127_RIGHT_CC.dcm
Update [0008|103e]: Calc-Test_P_00180_LEFT_CC.dcm
Update [0020|0060]: Calc-Test_P_00180_LEFT_CC.dcm
Update [0008|103e]: Calc-Test_P_00180_LEFT_MLO.dcm
Update [0020|0060]: Calc-Test_P_00180_LEFT_MLO.dcm
Update [0008|103e]: Calc-Test_P_00202_RIGHT_MLO.dcm
Update [0020|0060]: Calc-Test_P_00202_RIGHT_MLO.dcm
Update [0008|103e]: Calc-Test_P_00202_RIGHT_CC.dcm
Update [0020|0060]: Calc-Test_P_00202_RIGHT_CC.dcm
Update [0008|103e]: Calc-Test_P_00038_RIGHT_CC.dcm
Update [0020|0060]: Calc-Test_P_00038_RIGHT_CC.dcm
Update [0

  0%|          | 0/223 [00:00<?, ?it/s]

Cleanse: 46.85585880279541 s


100%|██████████| 69/69 [00:24<00:00,  2.85it/s]


## Create Node

In [ ]:
localNode = Node( "localhost", 8000 ) # local server

# If local node is crashed for some reason, you can try to load your Node
# localNode = Node( 'localhost', 8000, node_uid )

## Add Models

In [ ]:
denseNet121 = densenet121(
    spatial_dims=2,
    in_channels=3,
    out_channels=4, 
    pretrained=True
)

denseNet121.class_layers = torch.nn.Sequential( 
    torch.nn.ReLU( inplace=True ), 
    torch.nn.AdaptiveAvgPool2d( output_size=1),
    torch.nn.Flatten(start_dim=1, end_dim=-1), 
    torch.nn.Linear(in_features=1024, out_features=4, bias=True),
    torch.nn.Softmax()
)

denseNet169 = densenet169(spatial_dims=2, in_channels=3, out_channels=4, pretrained=True)
denseNet169.class_layers = torch.nn.Sequential( 
    torch.nn.ReLU( inplace=True ), 
    torch.nn.AdaptiveAvgPool2d( output_size=1),
    torch.nn.Flatten(start_dim=1, end_dim=-1), 
    torch.nn.Linear(in_features=1664, out_features=4, bias=True),
    torch.nn.Softmax()
)

resnet50 = se_resnet50( spatial_dims=2, in_channels=3, num_classes=4, pretrained=True)
resnet101 = se_resnet101( spatial_dims=2, in_channels=3, num_classes=4, pretrained=True)
resnet152 = se_resnet152( spatial_dims=2, in_channels=3, num_classes=4, pretrained=True)
resnet50.last_linear = torch.nn.Sequential( 
    torch.nn.Linear( in_features=2048, out_features=4, bias=True ), 
    torch.nn.Softmax()
)
resnet101.last_linear = torch.nn.Sequential( 
    torch.nn.Linear( in_features=2048, out_features=4, bias=True ), 
    torch.nn.Softmax()
)
resnet152.last_linear = torch.nn.Sequential( 
    torch.nn.Linear( in_features=2048, out_features=4, bias=True ), 
    torch.nn.Softmax()
)

In [ ]:
localNode.add_model([
    Model(denseNet121, 'UIowa_denseNet121'), 
    Model(denseNet169, 'UIowa_denseNet169'), 
    Model(resnet50, 'UIowa_resnet50'), 
    Model(resnet101, 'UIowa_resnet101'), 
    Model(resnet152, 'UIowa_resnet152')
])

In [ ]:
print( localNode.model_bench) 

## Train models

In [ ]:
# helper loss function
def cross_entropy_with_onehot(input, target):
    _, labels = target.max(dim=1)
    return torch.nn.CrossEntropyLoss()(input, labels)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print( 'device:', device )

# temporary directory to save a snapshot
# # update to your local directory
temp_saveSnapshot_dir = '/local/vol00/home/bochoi/tmpModels/' 

histories = [None] * len(localNode.model_bench)
for idx in range( len(localNode.model_bench) ):
    print(localNode.model_bench[idx].name)
    mkdir(temp_saveSnapshot_dir)

    optimizer = torch.optim.Adam( localNode.model_bench[idx].net.parameters(), 5e-2) # 5e-3 was too big, doesn't train
    # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau( optimizer, mode='min', factor=0.1, patience=50)
    histories[idx] = localNode.model_bench[idx].train(
        dataset=train_ds,
        loss_function=cross_entropy_with_onehot,
        optimizer=optimizer,
        epochs=500,
        metrics=["AUC"],
        batch_size=16,
        device=device,
        validation_dataset=val_ds,
        dirpath=temp_saveSnapshot_dir, 
        earlystop={'patience':50, 'delta':0}
    )
    if len( os.listdir( temp_saveSnapshot_dir ) ) != 0:
        localNode.model_bench[idx].load_snapshot(temp_saveSnapshot_dir)
    
    # save model's prediction performance
    _, _ = localNode.model_bench[idx].predict( train_ds, section='train')
    _, _ = localNode.model_bench[idx].predict( val_ds, section='validation')
    _, _ = localNode.model_bench[idx].predict( test_ds, section='test')
    
    localNode.model_bench[idx].commit('initial commit')
    remove( temp_saveSnapshot_dir)
    
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

cols = 4
plt.figure( figsize=(25, 5) ) 

plt.subplot( 1, cols, 1 )
for model_history in histories:
    plt.plot( model_history['loss'] )
plt.title( 'training loss vs epoch' )
plt.legend( [model.name for model in localNode.model_bench] )
plt.xlabel('epochs')
plt.ylabel('loss')

plt.subplot( 1, cols, 2 )
for model_history in histories:
    plt.plot( model_history['val_loss'] )
plt.title( 'validation loss vs epoch' )
plt.legend( [model.name for model in localNode.model_bench] )
plt.xlabel('epochs')
plt.ylabel('loss')

plt.subplot( 1, cols, 3 )
for model_history in histories:
    plt.plot( model_history['auc'] )
plt.title( 'training auc vs epoch' )
plt.legend( [model.name for model in localNode.model_bench] )
plt.xlabel('epochs')
plt.ylabel('loss')

plt.subplot( 1, cols, 4 )
for model_history in histories:
    plt.plot( model_history['val_auc'] )
plt.title( 'validation auc vs epoch' )
plt.legend( [model.name for model in localNode.model_bench] )
plt.xlabel('epochs')
plt.ylabel('loss')

plt.plot()

In [ ]:
for model in localNode.model_bench:
    print( model.name )
    print( model.history )
    print()

## Start your server 

(communicate through Slack) 

In [ ]:
localNode.start()

## Send your models

In [ ]:
# connect to the peers
localNode.connect_to('Peer IP address', open port )
localNode.connect_to('Another peer IP address', open port)

In [ ]:
for peer_idx in range( len(localNode.peers_outbound) ):
    for model_idx in range( 5 ):
        print( 'Sending', localNode.model_bench[model_idx].name, 'to', localNode.peers_outbound[peer_idx] )
        localNode.send_model(peer_idx, model_index=model_idx)

In [ ]:
print(localNode.model_bench )

## Create ensemble after recieving models

In [ ]:
ensemble = localNode.create_ensemble( name='ensemble',
                          size=3, 
                          dataset=val_ds,
                          models=localNode.model_bench,
                          diversity_measure='gd', 
                          test_dataset=test_ds,
                          vote_method='majority'
                         )

In [ ]:
ensemble.create( percentage=0.25, 
               method='hill_climbing')

In [ ]:
ensemble.set_best_ensemble()
ensemble.predict(test_ds)

## Share the result to collaborators

In [ ]:
ensemble.save('path/to/save/the/ensembleResult')